In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from ipywidgets import GridBox, Image, Text, Layout, HTML
import pandas as pd
import pickle
import plotly.graph_objects as go
from pyrfume.odorants import smiles_to_image

In [8]:
# Load embeddings computed in static.ipynb
with open('data/gdb_umap.pkl', 'rb') as f:
    gdb_umap = pickle.load(f)
with open('data/pf_umap.pkl', 'rb') as f:
    pf_umap = pickle.load(f)

In [26]:
# Load SMILES <-> PubChem ID mappings for known molecules
smiles_to_cids = pd.read_csv('data/pyrfume-filtered-cids.txt', sep='\t', index_col=0, header=None, names=['SMILES', 'CID']).fillna(0).astype(int)

In [63]:
%%html
<style>
.my-label{
    background-color: white;
}
a {
  text-decoration: underline;
}

/* unvisited link */
a:link {
  color: red;
}

/* visited link */
a:visited {
  color: green;
}

/* mouse over link */
a:hover {
  color: hotpink;
}

/* selected link */
a:active {
  color: blue;
}
</style>

In [73]:
# Dynamic Embedding
hover_on = 1  

def plot(big_umap, known_umap, skip=10):
    big_umap = big_umap.iloc[::skip]
    known_umap = known_umap.iloc[::skip]
    big_umap = pd.concat([big_umap, known_umap])  # Required to make hovering over known_umap points work correctly.
    # The GDB scatter plot
    skip = 10
    big_scatter = go.Scatter(
        x=big_umap.loc[:, 0],
        y=big_umap.loc[:, 1],
        name='Possible Molecules',
        mode="markers",
        hoverinfo="text",
        opacity=0.5,
        marker={
            "size": 5,
            "line": {"width": 0.5, "color": "white"},
            "color": big_umap.loc[:, 'p'],
            "colorscale": 'magma',
            "colorbar": {'thickness': 20, 'title': 'p(Odorous)'},
        },
    )

    # The known odorants scatter plot
    known_scatter = go.Scattergl(
        x=known_umap.loc[:, 0],
        y=known_umap.loc[:, 1],
        name='Known Odorants',
        mode="markers",
        hoverinfo="text",
        opacity=1,
        marker={
            "size": 5,
            "line": {"width": 0.5, "color": "white"},
            "color": "blue",
        },
    )

    # The axes, etc.
    layout = go.Layout(
        xaxis={"type": "linear", "title": "", "showline": False, "showticklabels": False},
        yaxis={"type": "linear", "title": "", "showline": False, "showticklabels": False},
        margin={"l": 40, "b": 40, "t": 10, "r": 10},
        legend={"x": 0, "y": 1, 'font':{'size':15, 'color': 'white'}},
        hovermode="closest",
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(10,10,10,1)",
        width=1000,
        height=1000,
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        xaxis_zeroline=False,
        yaxis_zeroline=False
    )

    fig = go.FigureWidget(data=[big_scatter, known_scatter], layout=layout)
    fig.layout.hovermode = 'closest'

    # The 2D drawing of the molecule
    first_smiles = "CCCCO"
    image_widget = Image(
        value=smiles_to_image(first_smiles),
        layout=Layout(height="200px", width="200px", left='50px', top='-315px')
        )
    
    # The SMILES string
    smiles_widget = Text(
        value=first_smiles,
        placeholder=first_smiles,
        description='SMILES:',
        disabled=False,
        style = {'description_width': '57px'},
        layout=Layout(width='500px', left='50px', top='-314px')
    )
    smiles_widget.add_class('my-label')
    
    # The PubChem ID, if available
    cid_widget = HTML(
        value='',
        placeholder='',
        description='PubChem ID:',
        disabled=False,
        style = {'description_width': '85px'},
        layout=Layout(width='200px', left='50px', top='-314px')
    )
    cid_widget.add_class('my-label')

    def hover_fn(trace, points, state):
        """Callback to execute when hovering over a point"""
        try:
            ind = points.point_inds[0]
            if trace.name == 'Possible Molecules':
                use = big_umap
            else:
                use = known_umap
            smiles = use.index[ind]
            image_widget.value = smiles_to_image(smiles, size=200)
            smiles_widget.value = smiles
            try:
                cid = smiles_to_cids.loc[smiles]
                url = "https://pubchem.ncbi.nlm.nih.gov/compound/%d" % cid
                cid_widget.value = '<a href="%s" target="_blank"><p style=''>%d</p></a>' % (url, cid)
                #js = 'window.open("%s","_blank")' % url
                #cid_widget.value = '<button class="btn btn-success" onclick="%s">%d</button>' % (js, cid)
            except:
                cid_widget.value = 'N/A'
        except IndexError:
            pass
        except Exception as e:
            print(e)
                  
    def click_fn(trace, points, state):
        """Callback to execute when clicking on a point"""
        global hover_on
        if hover_on:
            fig.data[0].on_hover(None)
            hover_on = 0
        else:
            fig.data[0].on_hover(hover_fn)
            hover_on = 1
            
    fig.data[0].on_hover(hover_fn)
    fig.data[0].on_click(click_fn)
    canvas = GridBox([fig, image_widget, smiles_widget, cid_widget], layout={'gridgap': '0'})
    return canvas

plot(gdb_umap, pf_umap, skip=5)

GridBox(children=(FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': array([0…